# 第4章：分离数据与指令

- [课程](#lesson)
- [练习](#exercises)
- [示例游乐场](#example-playground)

## 环境准备

运行下方初始化单元以加载 API Key 并建立 `get_completion` 助手函数。

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
import json

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt,system=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## 课程

很多时候，我们不想每次都写完整提示，而是希望先写好**可复用的提示模板**，在提交给 Claude 之前，再用新增的输入数据进行替换。这在“任务固定、但每次数据不同”的场景尤其有用。

好消息是：我们可以**将提示的固定骨架与变量用户输入分离**，在发送给 Claude 之前，把用户输入替换进提示模板即可。

下面将一步步讲解如何编写可替换的提示模板，以及如何将用户输入注入模板。

### 示例

第一个示例中，我们让 Claude 充当动物叫声生成器。注意，提交给 Claude 的完整提示，其实就是把输入（这里是 “Cow”）替换进 `PROMPT_TEMPLATE`。打印完整提示时，你会看到通过 f-string 将 `ANIMAL` 占位符替换成了 “Cow”。

**提示：** 实际使用中，占位变量的命名并无强制要求。本例使用 `ANIMAL`，也可以叫 `CREATURE` 或 `A`。但通常建议使用更具体、贴切的变量名，这样即使没进行替换，模板也便于理解。务必保证你在模板 f-string 中用到的变量名与声明的一致。

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

为什么要分离并替换输入？因为**提示模板能简化重复性工作**。设想你搭了一个提示结构，允许第三方用户提交内容（本例中是动物名称）。这些用户无需看到完整提示，也不用自己撰写提示，他们只需填写变量。

此处我们用变量与 f-string 进行替换，你也可以使用 `format()` 方法。

**提示：** 模板可以包含任意数量的变量！

在使用这种替换变量时，非常重要的一点是：**确保 Claude 清楚变量与指令（或任务描述）的边界**。先看一个未将指令与变量分离的反例。

对人类读者而言，下方模板中变量的起止很清晰；但在完成替换后的完整提示里，这种边界会变得不明显。

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

在此例中，**Claude 会把 “Yo Claude” 误判为邮件正文的一部分**！你会看到它把改写的开头写成了 “Dear Claude”。在人眼看来，尤其在模板中，邮件的起止很清楚，但替换后就不那么清晰了。

如何解决？**用 XML 标签把输入包起来**！我们在下例中这么做了，你会看到输出里不再出现 “Dear Claude”。

[XML 标签](https://docs.anthropic.com/claude/docs/use-xml-tags) 是形如 `<tag></tag>` 的尖括号标签，成对出现：如 `<tag>` 为起始，`</tag>` 为结束。它们用于包裹内容，例如：`<tag>content</tag>`。

**提示：** 虽然 Claude 能识别多种分隔符与定界方式，但我们建议**优先使用 XML 标签作为分隔符**，因为 Claude 专门针对其作为提示组织机制进行了训练。除函数调用外，**没有任何“特殊魔法”的 XML 标签需要你使用来获得额外性能**；我们有意使 Claude 在这方面保持灵活与可定制。

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

再看一个 XML 标签能帮忙的例子：

在下方提示中，**Claude 错把指令与输入的边界理解反了**。由于格式上的误导，它把 `Each is about an animal, like rabbits` 误当成了列表的一部分，而填充 `SENTENCES` 的用户显然不希望如此。

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

要修正这一点，我们只需**用 XML 标签把用户输入的句子包起来**。即便在 `Each is about an animal, like rabbits` 前带有误导性的短横线，Claude 也能明确知道输入数据的起止范围。`

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

**注意：** 在这个“Each is about an animal” 的错误示例里，我们是特意加了短横线，来演示 Claude 会如何被误导。这说明一个重要道理：**细节很重要**！请**仔细检查提示中的错别字与语法问题**。Claude 对模式十分敏感（早期在微调前，模型本质上是文本预测器），当你犯错时它更易出错；当你表达清晰时它就更聪明；当你显得随意时它也更随意，等等。

如果你想在不更改上方内容的前提下试验课程中的提示，请滚动到页面底部，前往[示例游乐场](#example-playground)。

---

## 练习
- [练习 4.1 - 俳句主题](#exercise-41---haiku-topic)
- [练习 4.2 - 狗的问题（有错字）](#exercise-42---dog-question-with-typos)
- [练习 4.3 - 狗的问题 第2部分](#exercise-42---dog-question-part-2)

### 练习 4.1 - 俳句主题
将 `PROMPT` 改为模板，接收名为 `TOPIC` 的变量，并输出关于该主题的俳句。本练习旨在测试你对使用 f-string 进行变量模板化的理解。

In [ ]:
# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f""

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 想要提示？运行下方单元！

In [ ]:
print(hints.exercise_4_1_hint)

### 练习 4.2 - 狗的问题（有错字）
通过添加 XML 标签修复 `PROMPT`，使 Claude 得出正确答案。

尽量不要改动提示的其他部分。文本中的混乱与错字是故意保留的，旨在观察 Claude 面对这类问题的表现。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 想要提示？运行下方单元！

In [ ]:
print(hints.exercise_4_2_hint)

### 练习 4.3 - 狗的问题 第2部分
在**不**添加 XML 标签的前提下修复 `PROMPT`。请仅删除一到两个词。

与上面的练习一样，尽量不要改动提示的其他部分。此练习将展示 Claude 能够解析与理解怎样的语言变体。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 想要提示？运行下方单元！

In [ ]:
print(hints.exercise_4_3_hint)

### 恭喜！

如果你已经完成到目前为止的所有练习，可以进入下一章。祝你玩得开心！

---

## 示例游乐场

这里是自由试验区，你可以复用上面的示例提示并做修改，观察它如何影响 Claude 的输出。

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))